In [0]:
question=dbutils.widget.get('query')

In [0]:
import os

from databricks.vector_search.client import VectorSearchClient
workspace_url = os.environ.get("WORKSPACE_URL")
sp_client_id = os.environ.get("SP_CLIENT_ID")
sp_client_secret = os.environ.get("SP_CLIENT_SECRET")
vsc = VectorSearchClient(
    workspace_url=workspace_url,
    service_principal_client_id=sp_client_id,
    service_principal_client_secret=sp_client_secret
)
results = vsc.get_index("spectra_transaction","main.default.spectratransaction_index").similarity_search(
  query_text=question,
  columns=["Chunks"],
  num_results=5)
rest = results.get('result', {}).get('data_array', [])

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


In [0]:
chain_config = {
    "llm_model_serving_endpoint_name": "databricks-dbrx-instruct",
    "vector_search_endpoint_name": "spectra_transaction",  # the endoint we want to use for vector search
    "vector_search_index":"main.default.spectratransaction_index",
    "llm_prompt_template": """You are an assistant that answers questions. use the following pieces of retrieved context to answer the question. Some pieces of context may be irrelevant, in which case you should not use them to form the answer.Look between the sentences for answers because answers might not be explicitly mentioned.\n\nContext: {context}""",
}

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks
from operator import itemgetter

prompt = ChatPromptTemplate.from_messages(
    [  
        ("system", chain_config.get("llm_prompt_template")), # Contains the instructions from the configuration
        ("user", "{question}") #user's questions
    ]
)

# Our foundation model answering the final prompt
model = ChatDatabricks(
    endpoint=chain_config.get("llm_model_serving_endpoint_name"),
    extra_params={"temperature": 0.01, "max_tokens": 500}
)
answer = (prompt | model).invoke({'question': question, 'context': rest})

In [0]:
answer.content

'Based on the provided context, the status of Transaction ID "TXN1234567890" is not explicitly mentioned. However, we can infer that the transaction might have encountered an error since the "Error Message" field has a value of "error 404".'